In [1]:
using Distributions, Plots, Interact

HTML{String}("    <script class='js-collapse-script'>\n        var curMatch =\n            window.location.href\n            .match(/(.*?)\\/notebooks\\/.*\\.ipynb/);\n\n        curMatch = curMatch ||\n            window.location.href\n            .match(/(.*?)\\/apps\\/.*\\.ipynb/);\n\n        if ( curMatch ) {\n            \$('head').append('<base href=\"' + curMatch[1] + '/\">');\n        }\n    </script>\n")

HTML{String}("<script class='js-collapse-script' src='/assetserver/3c9f011c98c5a0e07ee9ac232239d92766fb3f4c-assets/webio/dist/bundle.js'></script>")

HTML{String}("<script class='js-collapse-script' src='/assetserver/3c9f011c98c5a0e07ee9ac232239d92766fb3f4c-assets/providers/ijulia_setup.js'></script>")

HTML{String}("  <script class='js-collapse-script'>\n    \$('.js-collapse-script').parent('.output_subarea').css('padding', '0');\n  </script>\n")

# Quarts Performance
## Notation
* $g$ – number of replicas
* $m$ – number of sensors
* $h$ – number of actuators
* $p$ – network drop probability

In [2]:
# Default parameter values from Quarts paper
p = 1e-3
m = 10
g = 1
h = 1

1

The Quarts protocol introduces a `collect` phase as part of its `collect_and_vote` procedure.
In this phase, all replicas exchange their measurements an complete them componentwise.
Thereby, the availability of measurements at each replica is increased _a priori_.
The effective _unavailability_ of a single measurement component at each replica (which equals the drop probability $p$ without collection), when measurements are exchanged between $g$ replicas, is given by
$$p_\text{collect} = p\bigl(1-(1-p)^2\bigr)^{g-1}$$

In [3]:
p_collect(p, g) = p*(1-(1-p)^2)^(g-1)

p_collect (generic function with 1 method)

The Quarts authors assume that a replica is only available if it received _sufficient_ measurements, where the number of sufficient measurements is not specified.
Assuming that $n$ out of $m$ measurements are required, the availability of a single replica _after_ the `collect` phase can be derived from the cumulative binomial distribution as the probability that up to $m-n$ measurements are missing:
$$p_\text{available} = P\bigl( X ≤ m-n \bigr), \quad X\sim B(m,p_\text{collect})$$

In [4]:
p_available(p, m, g, n) = cdf(Binomial(m,p_collect(p,g)),m-n)

p_available (generic function with 1 method)

The unavailability of a single replica after `collect` (without taking node failures into account) for 1 to 10 replicas and $n=m-1$ required measurements is given by

In [9]:
P = 1 .- p_available.(p,m,1:10,m)

10-element Array{Float64,1}:
 0.00995512 
 1.99898e-5 
 3.996e-8   
 7.98801e-11
 1.5965e-13 
 3.33067e-16
 0.0        
 0.0        
 0.0        
 0.0        

In [14]:
P

10-element Array{Float64,1}:
   0.00995512 
   1.99898e-5 
   3.996e-8   
   7.98801e-11
   1.5965e-13 
   3.33067e-16
 NaN          
 NaN          
 NaN          
 NaN          

In [21]:
gr()
@manipulate for min_components = 1:m
    P = 1 .- p_available.(p,m,1:10,min_components)
    P[P.==0] = NaN
    plot(P, yscale=:log10, ylim=(1e-14,1), xlabel="Number of replicas (g)", ylabel="Unavailability")
end

(div
  Widgets.Widget{:slider}(DataStructures.OrderedDict{Symbol,Any}(:changes=>Observables.Observable{Int64}("ob_48", 0, Any[WebIO.SyncCallback(WebIO.Scope("knockout-component-523392c9-1690-40a3-8d23-95855e67109f", WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["min_components"], Dict{Symbol,Any}(Pair{Symbol,Any}(:className, "interact "),Pair{Symbol,Any}(:style, Dict{Any,Any}(Pair{Any,Any}(:padding, "5px 10px 0px 10px")))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"text-align:right;width:18%"))), 2), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:max, 10),Pair{Symbol,Any}(:min, 1),Pair{Symbol,Any}(:attributes, Dict{Any,Any}(Pair{Any,Any}(:type, "range"),Pair{Any,Any}(Symbol("data-bind"), "numericValue: value, valueUpdate: 'input', event: {change : function () {this.changes(this.changes()+1)}}"))),Pair{Symbol,Any}(:step, 1),Pair{Symbol,Any}(:className, "slider slider is-fullwidth"),Pair{Symbol,Any}(:style, Dict{Any,Any}())), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"flex-grow:1; margin: 0 2%"))), 1), WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[WebIO.Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("data-bind"=>"text: displayedvalue"))), 0)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"width:18%"))), 1)], Dict{Symbol,Any}(Pair{Symbol,Any}(:attributes, Dict("style"=>"display:flex; justify-content:center; align-items:center;"))), 7), Dict{String,Tuple{Observables.Observable,Union{Bool, Void}}}(Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("changes", (Observables.Observable{Int64}(#= circular reference @-7 =#), nothing)),Pair{String,Tuple{Observables.Observable,Union{Bool, Void}}}("value", (Observables.Observable{Int64}("ob_47", 5, Any[WebIO.SyncCallback(WebIO.Scope(#= circular reference @-7 =#), WebIO.SyncCallback(WebIO.Scope(#= circular reference @-8 =#), WebIO.#37)), Observables.g]), nothing))), Set{String}(), nothing, Any["knockout"=>"/home/futterbn/.julia/v0.6/Knockout/src/../assets/knockout.js", "knockout_punches"=>"/home/futterbn/.julia/v0.6/Knockout/src/../assets/knockout_punches.js", "/home/futterbn/.julia/v0.6/InteractBase/src/../assets/all.js", "/home/futterbn/.julia/v0.6/InteractBase/src/../assets/style.css", "/home/futterbn/.julia/v0.6/InteractBulma/src/../assets/main.css"], Dict{Any,Any}(Pair{Any,Any}("_promises", Dict{Any,Any}(Pair{Any,Any}("importsLoaded", Any[WebIO.JSString("function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init : function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            })\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            })\n            ko.applyBindingsToNode(element, { value: stringified, valueUpdate: allBindings.get('valueUpdate')}, context);\n        }\n    };\n    var json_data = JSON.parse(\"{\\\"changes\\\":0,\\\"value\\\":5}\");\n    var self = this;\n    function AppViewModel() {\n        for (var key in json_data) {\n            var el = json_data[key];\n            this[key] = Array.isArray(el) ? ko.observableArray(el) : ko.observable(el);\n        }\n        \n        [this[\"displayedvalue\"]=ko.computed(functio

In [ ]:
gr()
x = 1:6
y = 1:m
P = [1 - p_available(p,m,g,n) for g in x, n in y]
P[P.==0] = NaN
surface(x, y, P, scale=:log10, zlim=(1e-15,1), xlabel="Number of replicas (g)", zlabel="Unavailability")


# Our System Model
## Controller Execution Model
In our system model, the availability of an unreplicated controller — and by extension that of an individual replica — is _independent_ of received measurements.
Therefore, using our controller execution model with Quarts can be expected to yield at least as high an availability.

__Task:__ Modify Quarts simulation

## Consistency Model
Quarts does not require state consistency explicitly.
However, replicas are only allowed to make progress after agreement on the input and "state label".
Therefore, a kind of "skippable" state consistency may be implemented.

__Task:__ Verify using simulation